# **Step 1 - Configuration and Imports**

In [ ]:
#%pip install sec-edgar-downloader
#%pip install pandas

In [1]:
from sec_edgar_downloader import Downloader
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import pandas as pd
import glob, os, json, re, time

EMAIL        = "ecradeck@syr.edu"
CLEAN_PATH   = "../data/ESG_data_cleaned.csv"
FILINGS_DIR  = "../data/sec-filings"
FILINGS_BASE = "../data/sec-filings/sec-edgar-filings"
OUTPUT_DIR   = "../data/extracted_sections"
CHECKPOINT   = "../data/download_checkpoint.json"

os.makedirs(FILINGS_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Config loaded.")

Config loaded.


# **STEP 2 - Helper Functions**

In [4]:
# DATA
def load_companies(path=CLEAN_PATH):
    """Load cleaned ESG CSV and return list of (ticker, cik) tuples."""
    df = pd.read_csv(path)
    df['ticker'] = df['ticker'].str.upper()
    df['cik'] = df['cik'].astype(str).str.zfill(10)
    companies = list(zip(df['ticker'], df['cik']))
    print(f"Loaded {len(companies)} companies from {path}")
    return companies


# CHECKPOINTING
def load_checkpoint(path=CHECKPOINT):
    """Return (success, failed) lists from checkpoint, or ([], []) if none."""
    if os.path.exists(path):
        with open(path) as f:
            data = json.load(f)
        print(f"Checkpoint found: {len(data['success'])} done, {len(data['failed'])} failed.")
        return data['success'], data['failed']
    return [], []

def save_checkpoint(success, failed, path=CHECKPOINT):
    """Persist current success/failed lists to JSON."""
    with open(path, 'w') as f:
        json.dump({'success': success, 'failed': failed}, f)


# DOWNLOADING
def download_filings(companies, email=EMAIL, filings_dir=FILINGS_DIR):
    """
    Download 10-K filings for a list of (ticker, cik) tuples.
    Skips tickers already in checkpoint, saves progress after each download.
    Returns (success, failed) lists of tickers.
    """
    success, failed = load_checkpoint()
    done = set(success + failed)
    remaining = [(t, c) for t, c in companies if t not in done]
    print(f"Companies to download: {len(remaining)} (skipping {len(done)} already checkpointed)")

    dl = Downloader("Syracuse University", email, download_folder=filings_dir)

    for ticker, cik in remaining:
        try:
            dl.get("10-K", cik, before=datetime(2022, 4, 1), limit=1)
            success.append(ticker)
            print(f"✓ {ticker} (CIK: {cik})")
        except Exception as e:
            failed.append(ticker)
            print(f"✗ {ticker} (CIK: {cik}): {e}")
        save_checkpoint(success, failed)
        time.sleep(0.5)

    print(f"\nSuccess: {len(success)} | Failed: {len(failed)}")
    return success, failed


# TEXT EXTRACTION
def load_filing_text(file_path):
    """Parse a full-submission.txt and return cleaned plain text."""
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()
    try:
        soup = BeautifulSoup(content, 'html.parser')
    except Exception:
        return None
    return ' '.join(soup.get_text().split())

def extract_section(text, pattern, next_pattern):
    """Take the second match to skip TOC, extract until next section begins."""
    matches = list(re.finditer(pattern, text, re.IGNORECASE))
    m = matches[1] if len(matches) >= 2 else (matches[0] if matches else None)
    if m is None:
        return None
    start = m.start()
    next_match = re.search(next_pattern, text[start + 100:], re.IGNORECASE)
    end = start + 100 + next_match.start() if next_match else len(text)
    return text[start:end]

def extract_all_sections(file_path):
    """Run all three extractors on a single filing file."""
    text = load_filing_text(file_path)
    if text is None:
        return {'business': None, 'risk_factors': None, 'mda': None}
    return {
        'business':     extract_section(text, r'ITEM\s*1\.\s*BUSINESS',          r'ITEM\s*1A\.'),
        'risk_factors': extract_section(text, r'ITEM\s*1A\.\s*RISK\s*FACTORS',   r'ITEM\s*1B\.'),
        'mda':          extract_section(text, r'ITEM\s*7\.\s*MANAGEMENT',         r'ITEM\s*7A\.')
    }


# PROCESSING
def process_ticker(ticker, cik, filings_base=FILINGS_BASE, output_dir=OUTPUT_DIR):
    """Extract sections for one company, checking both ticker and CIK folder. Saves to JSON."""
    files = (
        glob.glob(f"{filings_base}/{ticker}/10-K/*/full-submission.txt") or
        glob.glob(f"{filings_base}/{cik}/10-K/*/full-submission.txt")
    )
    out_path = os.path.join(output_dir, f"{ticker}.json")
    if os.path.exists(out_path):
        return ticker, 'ok', 3
    if not files:
        return ticker, 'no_file', 0
    sections = extract_all_sections(files[0])
    n = sum(v is not None for v in sections.values())
    if n == 0:
        return ticker, 'no_sections', 0
    with open(os.path.join(output_dir, f"{ticker}.json"), 'w') as f:
        json.dump({"ticker": ticker, "sections": sections}, f)
    return ticker, 'ok', n

def run_extraction(success, companies, max_workers=4):
    """
    Run parallel section extraction for all successfully downloaded companies.
    Returns (extract_success, extract_partial, extract_failed) lists.
    """
    cik_map = {t: c for t, c in companies}
    ext_success, ext_partial, ext_failed = [], [], []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_ticker, t, cik_map.get(t, '')): t for t in success}
        for future in as_completed(futures):
            ticker, status, n = future.result()
            if status in ('no_file', 'no_sections'):
                ext_failed.append(ticker)
                print(f"✗ {ticker}: {status}")
            elif n < 3:
                ext_partial.append(ticker)
                print(f"⚠ {ticker}: {n}/3 sections")
            else:
                ext_success.append(ticker)
                print(f"✓ {ticker}: 3/3 sections")

    print(f"\n{'='*40}")
    print(f"Full (3/3):  {len(ext_success)}")
    print(f"Partial:     {len(ext_partial)}")
    print(f"Failed:      {len(ext_failed)}")
    return ext_success, ext_partial, ext_failed


# EXTRACTION QUALITY CHECK
def run_extraction_quality_check(output_dir=OUTPUT_DIR):
    """Build and print a summary quality check dataframe from extracted JSON files."""
    records = []
    for fname in os.listdir(output_dir):
        if not fname.endswith(".json"):
            continue
        with open(os.path.join(output_dir, fname)) as f:
            data = json.load(f)
        s = data['sections']
        records.append({
            'ticker':         data['ticker'],
            'business_chars': len(s['business'])    if s['business']    else 0,
            'risk_chars':     len(s['risk_factors']) if s['risk_factors'] else 0,
            'mda_chars':      len(s['mda'])          if s['mda']         else 0,
            'sections_found': sum(v is not None for v in s.values())
        })

    audit_df = pd.DataFrame(records).sort_values('ticker').reset_index(drop=True)

    print("Section Size Summary (characters)")
    for col, label in [('business_chars','BUSINESS'),('risk_chars','RISK FACTORS'),('mda_chars','MD&A')]:
        s = audit_df[col][audit_df[col] > 0]
        print(f"\n{label}")
        print(f"  Mean:   {s.mean():>10,.0f}")
        print(f"  Median: {s.median():>10,.0f}")
        print(f"  Min:    {s.min():>10,.0f}")
        print(f"  Max:    {s.max():>10,.0f}")

    print(f"\nCoverage")
    for n in [3, 2, 1, 0]:
        print(f"  {n}/3 sections: {(audit_df['sections_found'] == n).sum()} companies")

    partial = audit_df[audit_df['sections_found'] < 3]
    if not partial.empty:
        print(f"\nPartial Extractions ({len(partial)})")
        print(partial[['ticker','sections_found','business_chars','risk_chars','mda_chars']].to_string(index=False))

    return audit_df

print("All helper functions loaded.")

All helper functions loaded.


# **STEP 3 - Download All Filings**

In [13]:
companies = load_companies()
success, failed = download_filings(companies)

Loaded 709 companies from ../data/ESG_data_cleaned.csv
Companies to download: 709 (skipping 0 already checkpointed)
✓ DIS (CIK: 0001744489)
✓ GM (CIK: 0001467858)
✓ GWW (CIK: 0000277135)
✓ MHK (CIK: 0000851968)
✓ LYV (CIK: 0001335258)
✓ LVS (CIK: 0001300514)
✓ CLX (CIK: 0000021076)
✓ AACG (CIK: 0001420529)
✓ AAL (CIK: 0000006201)
✓ AAME (CIK: 0000008177)
✓ AAOI (CIK: 0001158114)
✓ AAON (CIK: 0000824142)
✓ AAPL (CIK: 0000320193)
✓ AATC (CIK: 0000943034)
✓ AAWW (CIK: 0001135185)
✓ AADI (CIK: 0001422142)
✓ ABEO (CIK: 0000318306)
✓ ABNB (CIK: 0001559720)
✓ ABIO (CIK: 0000907654)
✓ ABMD (CIK: 0000815094)
✓ ABOS (CIK: 0001576885)
✓ ABSI (CIK: 0001672688)
✓ ABTX (CIK: 0001642081)
✓ ABUS (CIK: 0001447028)
✓ ABVC (CIK: 0001173313)
✓ ACAD (CIK: 0001070494)
✓ ACCD (CIK: 0001481646)
✓ ACET (CIK: 0001720580)
✓ ABCL (CIK: 0001703057)
✓ ABCB (CIK: 0000351569)
✓ ACHL (CIK: 0001830749)
✓ ACHC (CIK: 0001520697)
✓ ACIW (CIK: 0000935036)
✓ ACLS (CIK: 0001113232)
✓ ACIU (CIK: 0001651625)
✓ ACHV (CIK: 00009

***Download Results***

10-K filings were successfully downloaded for **all 709 companies (100%)** in the cleaned ESG dataset, pulling the most recent filing submitted before April 2022 to align with the 2022 ESG rating period. Initial ticker-based downloads resolved 603/709, with the remaining 106 recovered by retrying directly with CIK identifiers from the ESG dataset. The complete set of 709 filings are ready for section extraction and FinBERT embedding.

# **STEP 4 - Test Extraction Using MSFT**

In [15]:
files = (
    glob.glob(f"{FILINGS_BASE}/MSFT/10-K/*/full-submission.txt") or
    glob.glob(f"{FILINGS_BASE}/0000789019/10-K/*/full-submission.txt")
)

if not files:
    print("MSFT filing not found — run Cell 4 first.")
else:
    sections = extract_all_sections(files[0])
    print(f"Extracted {sum(v is not None for v in sections.values())}/3 sections\n")
    for name, text in sections.items():
        if text:
            print(f"{name.upper()}: {len(text):,} chars")
            print(f"  First 200: {text[:200]}")
            print(f"  Last  200: {text[-200:]}")
        else:
            print(f"{name.upper()}: NOT FOUND")
        print()

Extracted 3/3 sections

BUSINESS: 74,757 chars
  First 200: ITEM 1. BUSINESS GENERAL Embracing Our Future Microsoft is a technology company whose mission is to empower every person and every organization on the planet to achieve more. We strive to create local
  Last  200: stors. We encourage investors, the media, and others interested in Microsoft to review the information we post on the social media channels listed on our Investor Relations website. 21 PART I Item 1A 

RISK_FACTORS: 68,581 chars
  First 200: ITEM 1A. RISK FACTORS Our operations and financial results are subject to various risks and uncertainties, including those described below, that could adversely affect our business, financial conditio
  Last  200: loyment-related laws are interpreted and applied to our workforce practices may result in increased operating costs and less flexibility in how we meet our workforce needs. 35 PART I Item 1B, 2, 3, 4 

MDA: 55,571 chars
  First 200: ITEM 7. MANAGEMENT’S DISCUSSION AND A

# **STEP 5 - Extract Sections for All Companies**

In [16]:
ext_success, ext_partial, ext_failed = run_extraction(success, companies)

✗ GWW: no_sections
✓ MHK: 3/3 sections
✓ DIS: 3/3 sections
✓ GM: 3/3 sections
✗ AACG: no_file
✗ LVS: no_sections
✗ AAME: no_sections
⚠ CLX: 2/3 sections
✓ AAON: 3/3 sections
✓ AAPL: 3/3 sections
✓ AATC: 3/3 sections
✗ LYV: no_sections
✓ AAWW: 3/3 sections
✓ ABEO: 3/3 sections
✓ AADI: 3/3 sections
✓ ABNB: 3/3 sections
✓ ABMD: 3/3 sections
✓ AAL: 3/3 sections
✓ ABIO: 3/3 sections
✓ ABSI: 3/3 sections
✓ ABOS: 3/3 sections
✓ AAOI: 3/3 sections
✓ ABUS: 3/3 sections
✓ ACAD: 3/3 sections
✓ ABTX: 3/3 sections
✓ ABVC: 3/3 sections
✓ ACCD: 3/3 sections
✗ ACHL: no_file
✓ ACET: 3/3 sections
✓ ABCL: 3/3 sections
✓ ACIW: 3/3 sections
✗ ACIU: no_file
✓ ABCB: 3/3 sections
✓ ACMR: 3/3 sections
✓ ACOR: 3/3 sections
✓ ACLS: 3/3 sections
✓ ACHV: 3/3 sections
✓ ACRS: 3/3 sections
✓ ACT: 3/3 sections
✓ ACTG: 3/3 sections
✓ ACVA: 3/3 sections
✗ ADAG: no_file
✓ ACXP: 3/3 sections
✓ ADBE: 3/3 sections
✓ ADAP: 3/3 sections
✓ ACRX: 3/3 sections
✓ ACHC: 3/3 sections
✓ ADI: 3/3 sections
✓ ADIL: 3/3 sections
✓ ACNB

: 

In [3]:
import os
saved = [f for f in os.listdir(OUTPUT_DIR) if f.endswith('.json')]
print(f"Saved so far: {len(saved)}/709")

Saved so far: 365/709


In [5]:
companies = load_companies()
success, failed = load_checkpoint()
ext_success, ext_partial, ext_failed = run_extraction(success, companies, max_workers=2)

Loaded 709 companies from ../data/ESG_data_cleaned.csv
Checkpoint found: 709 done, 0 failed.
✓ GM: 3/3 sections
✓ DIS: 3/3 sections
✓ MHK: 3/3 sections
✗ GWW: no_sections
✗ LVS: no_sections
✓ CLX: 3/3 sections
✗ AACG: no_file
✓ AAL: 3/3 sections
✗ AAME: no_sections
✓ AAOI: 3/3 sections
✓ AAON: 3/3 sections
✓ AAPL: 3/3 sections
✓ AATC: 3/3 sections
✓ AAWW: 3/3 sections
✓ AADI: 3/3 sections
✓ ABEO: 3/3 sections
✓ ABNB: 3/3 sections
✓ ABIO: 3/3 sections
✓ ABMD: 3/3 sections
✓ ABOS: 3/3 sections
✓ ABSI: 3/3 sections
✓ ABTX: 3/3 sections
✓ ABUS: 3/3 sections
✓ ABVC: 3/3 sections
✓ ACAD: 3/3 sections
✓ ACCD: 3/3 sections
✓ ACET: 3/3 sections
✓ ABCL: 3/3 sections
✓ ABCB: 3/3 sections
✗ ACHL: no_file
✓ ACHC: 3/3 sections
✓ ACIW: 3/3 sections
✓ ACLS: 3/3 sections
✗ ACIU: no_file
✓ ACHV: 3/3 sections
✓ ACMR: 3/3 sections
✓ ACOR: 3/3 sections
✓ ACNB: 3/3 sections
✓ ACRS: 3/3 sections
✓ ACRX: 3/3 sections
✓ ACT: 3/3 sections
✓ ACTG: 3/3 sections
✓ ACVA: 3/3 sections
✗ ADAG: no_file
✓ ACXP: 3/3 sec

# **STEP 6 - Extraction Quality Check**

In [6]:
audit_df = run_extraction_quality_check()
audit_df

Section Size Summary (characters)

BUSINESS
  Mean:      604,613
  Median:     47,326
  Min:           200
  Max:    32,160,939

RISK FACTORS
  Mean:    1,212,059
  Median:     84,111
  Min:           101
  Max:    74,141,257

MD&A
  Mean:      779,739
  Median:     69,358
  Min:           100
  Max:    74,160,319

Coverage
  3/3 sections: 618 companies
  2/3 sections: 17 companies
  1/3 sections: 5 companies
  0/3 sections: 0 companies

Partial Extractions (22)
ticker  sections_found  business_chars  risk_chars  mda_chars
  AEMD               2               0      123473      36542
   AEY               2         2593723           0    2571562
   CLX               2         5832090           0    5710578
   COP               2               0       43874     124865
  CTRA               2               0       88203      60480
   DTE               2               0       35090      63647
   DVN               2               0       56873      62106
   EIX               1               

,ticker,business_chars,risk_chars,mda_chars,sections_found
0,A,67124,60432,105003,3
1,AADI,139046,365078,56073,3
2,AAL,7675,254737,122148,3
3,AAOI,50893,76402,66252,3
4,AAON,34828,22323,41247,3
...,...,...,...,...,...
635,ZION,30162,38362,152245,3
636,ZTS,25054,179242,313805,3
637,ZUO,31227,166495,52425,3
638,ZWS,34599,52982,67615,3
